## <u>  Practise Session Times Code </u> 

### <u>  Imports </u> 

In [294]:
import os
import pandas as pd
import re
from PyPDF2 import PdfReader

### <u>  Creating Practise Lap Times DataFrame </u> 
- _FYI, This should take about 11 mins._

In [454]:
file_list = []
for path, subdirs, files in os.walk('../Downloaded Data/'):
    for name in files:
        file_list.append(os.path.join(path, name))
file_list = [file for file in file_list if "practicesessionlaptimes" in file]

# Useful for testing
# file_list = ['../Downloaded Data/2021/Abu Dhabi/2021_22_uae_f1_p1_timing_firstpracticesessionlaptimes_v01.pdf']

practise_times = pd.DataFrame()
i = 1

for file in file_list:
    
    race_info = re.split('-',re.sub('.+Data/(\d{4})/([^/]+)/.+',r'\1-\2', file))
    race_year = race_info[0]
    race_name = race_info[1]
    if "firstpracticesessionlaptimes" in file:
        session_name = "Free Practise 1"
    if "secondpracticesessionlaptimes" in file:
        session_name = "Free Practise 2"
    if "thirdpracticesessionlaptimes" in file:
        session_name = "Free Practise 3"
        
    print(f'File {i} of {len(file_list)}   Year: {race_year}     Race: {race_name}   Session: {session_name}')    

    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"

    text = text.replace("\n","")
    text = re.sub('(\.\d{3})(\d+)',r'\1 \2', text)
    text = re.sub('(:\d{2})(\d+)',r'\1 \2', text) ###
    text = re.sub('(\d+)(\s{2,})',r'-\1 ', text)
    text = re.sub('(\d{1,2})([A-Za-z ]+NOTIME)',r'-\1 \2', text) ###
    text = re.split('-', text)
    text = [data for data in text if data]

    for data in text:
        data = re.sub("NOTIME", " ", data)
        data = re.sub("((\.\d{3})|(:\d{2}))\s*[a-zA-OQ-Z].+",r"\1",data)
        data = re.sub("(\d)(P)(\d)",r"\1 \2 \3", data)
        data = re.sub(" (\d{1,2} (P )?\d{1,2}:\d{2}(\.\d{3}|:\d{2}))",r"-\1", data) ###
        data = re.split("-", data)
        if len(data) > 1:
            driver_info = re.split(' ', data[0], maxsplit = 1)
            driver_name = driver_info[1].strip()
            driver_number = driver_info[0]
            driver_laptimes = data[1:]
            for lap in driver_laptimes:
                lap_pit = "P" if "P" in lap else ""
                lap = re.sub("P ","", lap)
                lap_info = re.split(' ', lap, maxsplit = 1)
                lap_number = int(lap_info[0])
                lap_time = lap_info[1].strip()
                temp_df = pd.DataFrame({"Race Year":[race_year], "Race Name":[race_name], "Session Name":[session_name], "Driver Name":[driver_name], "Driver Number":[driver_number], "Lap Number":[lap_number], "Lap Time":[lap_time], "Pit":[lap_pit]})
                practise_times = practise_times.append(temp_df)
    i += 1            

File 1 of 161   Year: 2019     Race: Belgian   Session: Free Practise 1
File 2 of 161   Year: 2019     Race: Belgian   Session: Free Practise 2
File 3 of 161   Year: 2019     Race: Belgian   Session: Free Practise 3
File 4 of 161   Year: 2019     Race: Abu Dhabi   Session: Free Practise 1
File 5 of 161   Year: 2019     Race: Abu Dhabi   Session: Free Practise 2
File 6 of 161   Year: 2019     Race: Abu Dhabi   Session: Free Practise 3
File 7 of 161   Year: 2019     Race: Japanese   Session: Free Practise 1
File 8 of 161   Year: 2019     Race: Japanese   Session: Free Practise 2
File 9 of 161   Year: 2019     Race: Australian   Session: Free Practise 2
File 10 of 161   Year: 2019     Race: Australian   Session: Free Practise 1
File 11 of 161   Year: 2019     Race: Australian   Session: Free Practise 3
File 12 of 161   Year: 2019     Race: German   Session: Free Practise 1
File 13 of 161   Year: 2019     Race: German   Session: Free Practise 2
File 14 of 161   Year: 2019     Race: German 

In [419]:
times

,Race Year,Race Name,Session Name,Driver Name,Driver Number,Lap Number,Lap Time,Pit
0,2019,Belgian,FP1,Daniel RICCIARDO,3,1,11:04:02,P
0,2019,Belgian,FP1,Daniel RICCIARDO,3,2,20:55.949,
0,2019,Belgian,FP1,Daniel RICCIARDO,3,3,1:46.790,
0,2019,Belgian,FP1,Daniel RICCIARDO,3,4,2:15.317,
0,2019,Belgian,FP1,Daniel RICCIARDO,3,5,2:21.768,
...,...,...,...,...,...,...,...,...
0,2020,Styrian,FP1,Robert KUBICA,8,11,1:31.123,P
0,2020,Styrian,FP1,Robert KUBICA,8,12,10:52.780,
0,2020,Styrian,FP1,Robert KUBICA,8,13,1:06.797,
0,2020,Styrian,FP1,Robert KUBICA,8,14,1:38.423,
